In [ ]:
import sympy as sp
from sympy import init_printing
from IPython.display import display, Markdown, Latex
init_printing(use_latex='mathjax')

def print_basis(title, basis_vectors, dimension, extra_info=None):
    """
    Print basis vectors in a formatted way using SymPy LaTeX rendering
    
    Args:
        title (str): Title of the subspace (e.g., "ROW SPACE C(A^T) BASIS")
        basis_vectors (list): List of SymPy matrices representing basis vectors
        dimension (int): Dimension of the subspace
        extra_info (dict): Optional dictionary with additional info to display
    """
    display(Markdown(f"**{title}**"))
    
    if extra_info:
        for key, value in extra_info.items():
            display(Markdown(f"**{key}:** {value}"))
    
    # Display all basis vectors as columns (standard mathematical convention)
    if basis_vectors:
        # Convert all vectors to column vectors and place them side by side
        basis_matrix = None
        
        for vec in basis_vectors:
            # Convert to column vector if it's a row vector
            if vec.rows == 1:
                col_vec = vec.T  # Transpose row vector to column vector
            else:
                col_vec = vec
            
            # Build the basis matrix
            if basis_matrix is None:
                basis_matrix = col_vec
            else:
                basis_matrix = basis_matrix.row_join(col_vec)
        
        display(Markdown("**Basis vectors:**"))
        display(basis_matrix)
    
    display(Markdown(f"**Dimension:** {dimension}"))

# Basis of Four Fundamental Subspaces of a Matrix

There are 4 fundamental subspaces of a matrix
1. Row Space or Column Space of $A^T$ -> $C(A^T)$
2. Column Space -> C(A)
3. Null Space -> N(A)
4. Left Null Space or Null Space of $A^T$ -> $N(A^T)$

The approach of finding basis for all vectors involves transforming the matrix into reduced row echelon form (RREF). We use Gaussian elimination for that

##  Worked out procedure

### Gaussian Elimination 

#### Original Matrix

In [ ]:
A = sp.Matrix([[1, 2, 0, 2, 1], 
     [3, 6, 1, 9, 6], 
     [2, 4, 1, 7, 5]]) 
R= A
display(R)

⎡1  2  0  2  1⎤
⎢             ⎥
⎢3  6  1  9  6⎥
⎢             ⎥
⎣2  4  1  7  5⎦

Let us augment this by an identity matrix so that it is easy to find basis for left null space

In [ ]:
R = R.row_join(sp.eye(A.rows))

#### First Pivot  
Multiply and subtract first row from the below rows to make the entries below 0

R2 = R2 - 3 * R1 
R3 = R3 - 2 * R1

In [ ]:
R[1,:] = R[1,:] - 3*R[0,:]
R[2,:] = R[2,:] - 2*R[0,:]
display(R)

⎡1  2  0  2  1  1   0  0⎤
⎢                       ⎥
⎢0  0  1  3  3  -3  1  0⎥
⎢                       ⎥
⎣0  0  1  3  3  -2  0  1⎦

#### Second Pivot
Multiply and subtract second row from the below rows to make the entries below 0

R3 = R3 -R2

In [ ]:
R[2,:] = R[2,:] - R[1,:]
display(R)

⎡1  2  0  2  1  1   0   0⎤
⎢                        ⎥
⎢0  0  1  3  3  -3  1   0⎥
⎢                        ⎥
⎣0  0  0  0  0  1   -1  1⎦

This is already in RREF form and no further reduction is required

### Row Space , $C(A^T)$

Non zero rows of R forms the basis

Basis of $C(A^T)$ = 

In [ ]:
rb1 = R[0,:-A.rows]
rb2 = R[1,:-A.rows]

# Print row space basis using the reusable function
print_basis("ROW SPACE C(A^T) BASIS", [rb1, rb2], 2)

**ROW SPACE C(A^T) BASIS**

**Basis vectors:**

⎡1  0⎤
⎢    ⎥
⎢2  0⎥
⎢    ⎥
⎢0  1⎥
⎢    ⎥
⎢2  3⎥
⎢    ⎥
⎣1  3⎦

**Dimension:** 2

### Column Space  $C(A)$
Identify the columns in the RREF matrix, R, that contain the pivots. The basis consists of the corresponding columns from the original matrix, A.

In [ ]:
pidx = [0,2] # pivot columns
cb1 = A[:,pidx[0]]
cb2 = A[:,pidx[1]]

# Print column space basis using the reusable function
print_basis("COLUMN SPACE C(A) BASIS", [cb1, cb2], len(pidx), 
           extra_info={"Pivot columns": pidx})

**COLUMN SPACE C(A) BASIS**

**Pivot columns:** [0, 2]

**Basis vectors:**

⎡1  0⎤
⎢    ⎥
⎢3  1⎥
⎢    ⎥
⎣2  1⎦

**Dimension:** 2

### Null Space $N(A)$

This is the solution to $Ax = 0$, which can be found by solving $Rx = 0$. We can find free variables from RREF and express the solution in terms of that

In [ ]:
fidx = [1,3,4] # free columns

nb1 = sp.zeros(A.cols, len(fidx)) # null space basis vectors
for idx in range(len(fidx)):
    nb1[fidx[idx], idx] = 1  # Set free variable to 1
    nb1[pidx[0], idx] = -rb1[fidx[idx]]  # Set pivot variable based on first pivot row
    nb1[pidx[1], idx] = -rb2[fidx[idx]]  # Set pivot variable based on second pivot row

# Print null space basis using the reusable function
null_vectors = [nb1[:,i] for i in range(len(fidx))]
print_basis("NULL SPACE N(A) BASIS", null_vectors, len(fidx),
           extra_info={"Free variable columns": fidx, "Pivot columns": pidx})

**NULL SPACE N(A) BASIS**

**Free variable columns:** [1, 3, 4]

**Pivot columns:** [0, 2]

**Basis vectors:**

⎡-2  -2  -1⎤
⎢          ⎥
⎢1   0   0 ⎥
⎢          ⎥
⎢0   -3  -3⎥
⎢          ⎥
⎢0   1   0 ⎥
⎢          ⎥
⎣0   0   1 ⎦

**Dimension:** 3

### Left Null Space $N(A^T)$

This can be easily found from the augmented part of RREF by taking the rows corresponding to zero rows in RREF

In [ ]:
lnb1 = R[2, -A.rows:]

# Print left null space basis using the reusable function
print_basis("LEFT NULL SPACE N(A^T) BASIS", [lnb1], 1,
           extra_info={"Source": "Zero row in RREF"})

**LEFT NULL SPACE N(A^T) BASIS**

**Source:** Zero row in RREF

**Basis vectors:**

⎡1 ⎤
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣1 ⎦

**Dimension:** 1

## Using SymPy Built-in Methods

We can also solve this problem directly using SymPy's built-in methods for a quick verification:

In [ ]:
# Using SymPy built-in methods for verification

# Get RREF and pivot columns
rref_matrix, pivots = A.rref()

# Get row space basis: non-zero rows of RREF
row_space_vectors = []
for i in range(rref_matrix.rows):
    row = rref_matrix.row(i)
    if not row.equals(sp.zeros(1, rref_matrix.cols)):
        row_space_vectors.append(row)

# Get column space basis: columns of A corresponding to pivot columns
col_space_vectors = [A.col(i) for i in pivots]

# Get null space basis using SymPy's built-in method
null_space_vectors = A.nullspace()

# Get left null space basis: null space of A transpose
left_null_space_vectors = A.T.nullspace()


# --- Display Results using the existing print_basis function ---


# Use the existing print_basis function for consistent formatting
print_basis("ROW SPACE C(A^T) BASIS (SymPy)", row_space_vectors, len(row_space_vectors),
           extra_info={"Pivot columns": list(pivots)})

print_basis("COLUMN SPACE C(A) BASIS (SymPy)", col_space_vectors, len(col_space_vectors),
           extra_info={"Pivot columns": list(pivots)})

print_basis("NULL SPACE N(A) BASIS (SymPy)", null_space_vectors, len(null_space_vectors),
           extra_info={"Free variables": f"{A.cols - len(pivots)} variables"})

print_basis("LEFT NULL SPACE N(A^T) BASIS (SymPy)", left_null_space_vectors, len(left_null_space_vectors),
           extra_info={"Verification": "Null space of transpose"})

**ROW SPACE C(A^T) BASIS (SymPy)**

**Pivot columns:** [0, 2]

**Basis vectors:**

⎡1  0⎤
⎢    ⎥
⎢2  0⎥
⎢    ⎥
⎢0  1⎥
⎢    ⎥
⎢2  3⎥
⎢    ⎥
⎣1  3⎦

**Dimension:** 2

...

**LEFT NULL SPACE N(A^T) BASIS (SymPy)**

**Verification:** Null space of transpose

**Basis vectors:**

⎡1 ⎤
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣1 ⎦

**Dimension:** 1

## Using SVD

In [ ]:
# Using SymPy SVD to find the four fundamental subspaces

# Compute SVD using SymPy: A = U * Σ * V^T
U, sigma_vector, V = A.singular_value_decomposition()

# Display SVD components

display(Markdown("**U matrix:**"))
display(U)

display(Markdown("**Singular values (diagonal of Σ):**"))
# Convert singular values to a list for display
sigma_list = [sigma_vector[i] for i in range(len(sigma_vector))]
display(Markdown(f"σ = {sigma_list}"))

display(Markdown("**V matrix:**"))
display(V)

# Determine rank from non-zero singular values
rank = len([s for s in sigma_vector if s != 0])

display(Markdown(f"**Matrix rank from SVD:** {rank}"))

# Extract the four fundamental subspaces from SVD

# 1. Column Space C(A): First 'rank' columns of U
col_space_svd = [U.col(i) for i in range(rank)]

# 2. Left Null Space N(A^T): Last (m-rank) columns of U
left_null_space_svd = [U.col(i) for i in range(rank, U.cols)]

# 3. Row Space C(A^T): First 'rank' rows of V^T (which are first 'rank' columns of V)  
row_space_svd = [V.col(i) for i in range(rank)]

# 4. Null Space N(A): Last (n-rank) rows of V^T (which are last (n-rank) columns of V)
null_space_svd = [V.col(i) for i in range(rank, V.cols)]

# Display results using the existing print_basis function
print_basis("COLUMN SPACE C(A) BASIS (SVD)", col_space_svd, len(col_space_svd),
           extra_info={"Method": "First r columns of U matrix", "Rank": rank})

print_basis("ROW SPACE C(A^T) BASIS (SVD)", row_space_svd, len(row_space_svd),
           extra_info={"Method": "First r columns of V matrix", "Rank": rank})

print_basis("NULL SPACE N(A) BASIS (SVD)", null_space_svd, len(null_space_svd),
           extra_info={"Method": "Last (n-r) columns of V matrix", "Nullity": len(null_space_svd)})

print_basis("LEFT NULL SPACE N(A^T) BASIS (SVD)", left_null_space_svd, len(left_null_space_svd),
           extra_info={"Method": "Last (m-r) columns of U matrix", "Dimension": len(left_null_space_svd)})

**U matrix:**

⎡               √61   5                               5   √61              ⎤
⎢             - ─── - ─                             - ─ + ───              ⎥
⎢                9    9                               9    9               ⎥
⎢────────────────────────────────────  ────────────────────────────────────⎥
⎢     _______________________________       _______________________________⎥
⎢    ╱            2                2       ╱            2                2 ⎥
⎢   ╱  ⎛  4   √61⎞        ⎛5   √61⎞       ╱  ⎛  5   √61⎞        ⎛4   √61⎞  ⎥
⎢  ╱   ⎜- ─ + ───⎟  + 1 + ⎜─ + ───⎟      ╱   ⎜- ─ + ───⎟  + 1 + ⎜─ + ───⎟  ⎥
⎢╲╱    ⎝  9    9 ⎠        ⎝9    9 ⎠    ╲╱    ⎝  9    9 ⎠        ⎝9    9 ⎠  ⎥
⎢                                                                          ⎥
⎢              4   √61                               4   √61               ⎥
⎢              ─ - ───                               ─ + ───               ⎥
⎢              9    9                                9    9                ⎥

**Singular values (diagonal of Σ):**

σ = [sqrt(134 - 17*sqrt(61)), 0, 0, sqrt(17*sqrt(61) + 134)]

**V matrix:**

...

**Dimension:** 0

**LEFT NULL SPACE N(A^T) BASIS (SVD)**

**Method:** Last (m-r) columns of U matrix

**Dimension:** 0

**Dimension:** 0